In [0]:
#sensitivity SVM
#https://archive.ics.uci.edu/ml/datasets/Acute+Inflammations

In [0]:
!git clone https://github.com/marcingrzyb/computational-intelligence.git

Cloning into 'computational-intelligence'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [0]:
with open('/content/computational-intelligence/project1/diagnosis.data', encoding = 'utf-16') as fin, open('/content/computational-intelligence/project1/diagnosis2.data','w+') as fout:
    fout.write('a1;a2;a3;a4;a5;a6;d1;d2\n')
    for line in fin:
        line=line.replace(',','.')
        fout.write(line.replace('\t', ';'))



In [0]:
import pandas as pd
import numpy as np
df1=pd.read_csv('/content/computational-intelligence/project1/diagnosis2.data',sep=';')

df=df1.replace('yes',True).replace('no',False)

d1=df.pop('d1') #decision1
d2=df.pop('d2')#decision2
d1.name='Decision 1'
d2.name='Decision 2'

In [0]:
print(df)

       a1     a2     a3     a4     a5     a6
0    35.5  False   True  False  False  False
1    35.9  False  False   True   True   True
2    35.9  False   True  False  False  False
3    36.0  False  False   True   True   True
4    36.0  False   True  False  False  False
..    ...    ...    ...    ...    ...    ...
115  41.4  False   True   True  False   True
116  41.5  False  False  False  False  False
117  41.5   True   True  False   True  False
118  41.5  False   True   True  False   True
119  41.5  False   True   True  False   True

[120 rows x 6 columns]


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

def getData(df,results,dType):
  dataFrame=df.copy(deep=True)
  if(dType=='normalized') :
    dataFrame[['a1']]=MinMaxScaler().fit_transform(dataFrame[['a1']])
    X_train, X_test, y_train, y_test = train_test_split(
      df, results, test_size=0.20, random_state=41,stratify=results)#split to train and test
    X_train, X_valid, y_train, y_valid = train_test_split(
      X_train, y_train, test_size=0.5, random_state=41, stratify=y_train)#split train to train and validation
    return X_train,X_valid,X_test,y_train,y_test,y_valid
  elif(dType=='standarized') :
    print(dType)
    dataFrame[['a1']]=StandardScaler().fit_transform(dataFrame[['a1']])
    X_train, X_test, y_train, y_test = train_test_split(
      df, results, test_size=0.20, random_state=41, stratify=results)#split to train and test
    X_train, X_valid, y_train, y_valid = train_test_split(
      X_train, y_train, test_size=0.5, random_state=41, stratify=y_train)#split train to train and validation
    return X_train,X_valid,X_test,y_train,y_test,y_valid
  elif(dType== 'pca1') :
    dataFrame[['a1']]=PCA(n_components=0.5).fit_transform(dataFrame[['a1']])
    #dataFrame=dataFrame.pop('a1') #because PCA with n_components=0 returns []
    X_train, X_test, y_train, y_test = train_test_split(
      df, results, test_size=0.20, random_state=41,stratify=results)#split to train and test
    X_train, X_valid, y_train, y_valid = train_test_split(
      X_train, y_train, test_size=0.5, random_state=41,stratify=y_train)#split train to train and validation
    return X_train,X_valid,X_test,y_train,y_test,y_valid    
  elif(dType=='pca2') :
    dataFrame[['a1']]=PCA(n_components=1).fit_transform(dataFrame[['a1']])
    X_train, X_test, y_train, y_test = train_test_split(
      df, results, test_size=0.20, random_state=41,stratify=results)#split to train and test
    X_train, X_valid, y_train, y_valid = train_test_split(
      X_train, y_train, test_size=0.5, random_state=41, stratify=y_train)#split train to train and validation
    return X_train,X_valid,X_test,y_train,y_test,y_valid        
  else:
    X_train, X_test, y_train, y_test = train_test_split(
      df, results, test_size=0.20, random_state=41, stratify=results)#split to train and test
    X_train, X_valid, y_train, y_valid = train_test_split(
      X_train, y_train, test_size=0.5, random_state=41, stratify=y_train)#split train to train and validation
    return X_train,X_valid,X_test,y_train,y_test,y_valid      



In [0]:
from sklearn import svm, model_selection

def svcGridSearch(X, y, nfolds,C,gamma):
    param_grid = {'C': C, 'gamma' : gamma}
    grid_search = model_selection.GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds,scoring='balanced_accuracy')
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [0]:
Cs = [0.001, 0.01, 0.1,1,10]
gammas = [0.001, 0.01, 0.1,1,10]

In [0]:
from sklearn.metrics import confusion_matrix
def prepareRaport(dataFrame,decision,Cs_,gammas_):
  print(decision.name+': ')
  X_train,X_valid,X_test,y_train,y_test,y_valid=getData(dataFrame,decision,'basic')

  bestParams=svcGridSearch(X_valid,y_valid,10,Cs_,gammas_)
  clf = svm.SVC(C=bestParams['C'],gamma=bestParams['gamma'])
  clf.fit(X_train,y_train)

  y_pred=clf.predict(X_test)
  cm=confusion_matrix(y_test, y_pred)
  print(cm)

  sensitivity = float(cm[0][0])/np.sum(cm[0])

  print(sensitivity)

In [0]:
prepareRaport(df,d1,Cs,gammas)
prepareRaport(df,d2,Cs,gammas)

Decision 1: 
[[12  0]
 [ 0 12]]
1.0
Decision 2: 
[[14  0]
 [ 0 10]]
1.0


In [0]:
from sklearn.model_selection import cross_val_score
Cs = [0.001, 0.01, 0.1,0.5]
gammas = [0.001, 0.01, 0.1,10]
dataTypes=['basic','normalized','standarized','pca1','pca2']
def getBestModel(dataFrame,decision):
  results=[]
  for dType in dataTypes:
    X_train,X_valid,X_test,y_train,y_test,y_valid=getData(df,d1,dType)
    for C in Cs:
      for gamma in gammas:
        results.append((cross_val_score(estimator=svm.SVC(C=C,gamma=gamma), X=X_valid, y=y_valid, cv=10,scoring='balanced_accuracy').mean(),C,gamma,dType))
  return results

In [0]:
#Decision 1
result=getBestModel(df,d1)
print(np.array(result))
bestParams=max(result)
print(bestParams)

standarized
[['0.5' '0.001' '0.001' 'basic']
 ['0.5' '0.001' '0.01' 'basic']
 ['0.5' '0.001' '0.1' 'basic']
 ['0.5' '0.001' '10' 'basic']
 ['0.5' '0.01' '0.001' 'basic']
 ['0.5' '0.01' '0.01' 'basic']
 ['0.5' '0.01' '0.1' 'basic']
 ['0.5' '0.01' '10' 'basic']
 ['0.5' '0.1' '0.001' 'basic']
 ['0.5' '0.1' '0.01' 'basic']
 ['0.5' '0.1' '0.1' 'basic']
 ['0.5' '0.1' '10' 'basic']
 ['0.5' '0.5' '0.001' 'basic']
 ['0.5' '0.5' '0.01' 'basic']
 ['0.8666666666666666' '0.5' '0.1' 'basic']
 ['0.8833333333333332' '0.5' '10' 'basic']
 ['0.5' '0.001' '0.001' 'normalized']
 ['0.5' '0.001' '0.01' 'normalized']
 ['0.5' '0.001' '0.1' 'normalized']
 ['0.5' '0.001' '10' 'normalized']
 ['0.5' '0.01' '0.001' 'normalized']
 ['0.5' '0.01' '0.01' 'normalized']
 ['0.5' '0.01' '0.1' 'normalized']
 ['0.5' '0.01' '10' 'normalized']
 ['0.5' '0.1' '0.001' 'normalized']
 ['0.5' '0.1' '0.01' 'normalized']
 ['0.5' '0.1' '0.1' 'normalized']
 ['0.5' '0.1' '10' 'normalized']
 ['0.5' '0.5' '0.001' 'normalized']
 ['0.5' '0.5

In [0]:
X_train,X_valid,X_test,y_train,y_test,y_valid=getData(df,d1,bestParams[3])
clf = svm.SVC(C=bestParams[1],gamma=bestParams[2])
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

sensitivity = float(cm[0][0])/np.sum(cm[0])

print(sensitivity)

standarized
[[12  0]
 [ 4  8]]
1.0


In [0]:
#Decision 2
result=getBestModel(df,d2)
print(np.array(result))
bestParams=max(result)
print(bestParams)

standarized
[['0.5' '0.001' '0.001' 'basic']
 ['0.5' '0.001' '0.01' 'basic']
 ['0.5' '0.001' '0.1' 'basic']
 ['0.5' '0.001' '10' 'basic']
 ['0.5' '0.01' '0.001' 'basic']
 ['0.5' '0.01' '0.01' 'basic']
 ['0.5' '0.01' '0.1' 'basic']
 ['0.5' '0.01' '10' 'basic']
 ['0.5' '0.1' '0.001' 'basic']
 ['0.5' '0.1' '0.01' 'basic']
 ['0.5' '0.1' '0.1' 'basic']
 ['0.5' '0.1' '10' 'basic']
 ['0.5' '0.5' '0.001' 'basic']
 ['0.5' '0.5' '0.01' 'basic']
 ['0.8666666666666666' '0.5' '0.1' 'basic']
 ['0.8833333333333332' '0.5' '10' 'basic']
 ['0.5' '0.001' '0.001' 'normalized']
 ['0.5' '0.001' '0.01' 'normalized']
 ['0.5' '0.001' '0.1' 'normalized']
 ['0.5' '0.001' '10' 'normalized']
 ['0.5' '0.01' '0.001' 'normalized']
 ['0.5' '0.01' '0.01' 'normalized']
 ['0.5' '0.01' '0.1' 'normalized']
 ['0.5' '0.01' '10' 'normalized']
 ['0.5' '0.1' '0.001' 'normalized']
 ['0.5' '0.1' '0.01' 'normalized']
 ['0.5' '0.1' '0.1' 'normalized']
 ['0.5' '0.1' '10' 'normalized']
 ['0.5' '0.5' '0.001' 'normalized']
 ['0.5' '0.5

In [0]:
X_train,X_valid,X_test,y_train,y_test,y_valid=getData(df,d2,bestParams[3])
clf = svm.SVC(C=bestParams[1],gamma=bestParams[2])
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

sensitivity = float(cm[0][0])/np.sum(cm[0])

print(sensitivity)

standarized
[[14  0]
 [ 0 10]]
1.0
